**19/12/2021**
<br>
Bruno Mazzilli González

> # EDA: parte 1

En este notebook se van preparar los dataframes para poder empezar a analizarlos

## Librerias

In [1]:
import numpy as np
import pandas as pd
#import matplotlib
#from matplotlib import pyplot as plt # plots
#import seaborn as sns # plots
#from sklearn.ensemble import IsolationForest
#from sklearn import preprocessing 
#from sklearn.preprocessing import MinMaxScaler

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Importamos datos

In [3]:
data = pd.read_csv('data/cca_df.csv', low_memory=False) #low_memory = False es para evitar el warning por ser columnas de distinto formato

In [4]:
data.head(3)

,Unnamed: 0,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,0,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,...,1.0,6.0,1990.0,1.0,M,41.0,11.0,1.0,NaN,1.0
1,1,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,...,2.0,1.0,1987.0,1.0,M,19.0,11.0,1.0,NaN,1.0
2,2,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,...,2.0,1.0,1987.0,2.0,F,20.0,13.0,2.0,2.0,2.0


<br>
Eliminamos la columna de índice 'Unnamed: 0', que se crea por defecto

In [5]:
# Al crear el data frame python exporta el csv añadiendo la columna del índice
del data['Unnamed: 0']

In [6]:
data.head(3)

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,5.0,...,1.0,6.0,1990.0,1.0,M,41.0,11.0,1.0,NaN,1.0
1,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,5.0,...,2.0,1.0,1987.0,1.0,M,19.0,11.0,1.0,NaN,1.0
2,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,5.0,...,2.0,1.0,1987.0,2.0,F,20.0,13.0,2.0,2.0,2.0


In [7]:
data.shape

(5860405, 22)

<br>

## ESTUDIO DE VARIABLES:
### Variable Target:
La variable objetivo del estudio que se quiere predecir es: ¿habrá fallecimientos o no? En el data set original esta información la proporciona la variables: "C_SEV"

A continuacion se analiza cual es la frecuencia de cada uno de los posibles valores de las variables "C_SEV" 

### Variable Target: Fallecimiento

In [8]:
data['C_SEV'].isna().value_counts()

False    5860405
Name: C_SEV, dtype: int64

In [9]:
data['C_SEV'].value_counts(dropna=False)

2    5761772
1      98633
Name: C_SEV, dtype: int64

En primer lugar analizamos los accidentes que han resultado en algun fallecimiento

En función de los valores de la variable "C_SEV", se definen dos grupos:
Grupo 1: El accidente a resultado en almenos un fallecido
Grupo 2: El accidente a resultado en una lesion no mortal

Observamos que hay **98.633** accidentes con algun fallecimiento y **5.761.772** con alguna lesión no mortal.
Nuestros datos estan desbalanceados en la distribucion de los grupos 1 y 2. 
Para evitar que este 'bias' en el training set afecte a los algoritmos que usaremos tendremos que rebalancear los datos mas adelante

In [10]:
#creamos un dataframe especifico para esta variable target.
c_sev_df = data
# Creamos columna asociada con la variable target. toma valor 1 si ha habido algun fallecido y 0 en caso contrario
c_sev_df['target_c_sev'] = np.where((c_sev_df['C_SEV'] == 1.0), 1, 0)
#eliminamos la columna 'C_SEV' ya que ya tenemos una columna asociada con la variable target
del c_sev_df['C_SEV']
#excluimos la variable P_ID y V_ID ya que se considera el numero de identificacion de una persona no aporta informacion a 
#la variable target
del c_sev_df['P_ID']
del c_sev_df['V_ID']

In [11]:
print("Nuestro dataframe para el Target Fallecimiento tiene la forma:", c_sev_df.shape)

Nuestro dataframe para el Target Fallecimiento tiene la forma: (5860405, 20)


In [12]:
c_sev_df.head(3)

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,V_YEAR,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,target_c_sev
0,1999,1.0,1.0,20.0,2.0,34.0,NaN,1.0,5.0,3.0,3.0,6.0,1990.0,M,41.0,11.0,1.0,NaN,1.0,0
1,1999,1.0,1.0,20.0,2.0,34.0,NaN,1.0,5.0,3.0,3.0,1.0,1987.0,M,19.0,11.0,1.0,NaN,1.0,0
2,1999,1.0,1.0,20.0,2.0,34.0,NaN,1.0,5.0,3.0,3.0,1.0,1987.0,F,20.0,13.0,2.0,2.0,2.0,0


<br>

### Tratamiento de NAs

Ahora que ya tenemos el dataframe asociado a nuestra variable target procedemos a el tratamiento de valores missing

In [13]:
#creamos una funcion que identifique los valores NA en cada una de las variable de un dataframe

def identifica_NA(data):
    """
    Esta funcion devuelve un dataframe con el numero total de NAs y el porcentaje de NAs en cada columna.
    
    Parametros
    ----------
    data: dataframe
    """
    #creamos serie de pandas que cuente el numero de NAs para cada columna:
    null_sum = data.isnull().sum()
    #ponemos en orden de mas NAs a menos:
    total = null_sum.sort_values(ascending=False)
    #porcentaje de NAs
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False) #porcentaje de NAs
    
    # concatenamos columnas 'total' y 'percent' para crear el dataframe:
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    # eliminamos filas que de este dataframe que no tengan valores NA. omitir si quieres saber que variables no tienen NAs
    #df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

#### Tratamiento NAs dataframe con Variable Target: Fallecimiento

In [14]:
identifica_NA(c_sev_df)

,Number of NA,Percent NA
P_SAFE,1234284,21.06
C_RCFG,648946,11.07
V_YEAR,584378,9.97
C_CONF,463999,7.92
C_RALN,463312,7.91
P_AGE,395156,6.74
P_ISEV,371412,6.34
C_TRAF,305501,5.21
V_TYPE,283111,4.83
P_SEX,249740,4.26


Observando el numero de NAs en las columnas de nuestro dataframe podemos ver que no podemos simplemente eliminar todas las filas con NAs ya que esto eliminaría un gran porcentaje de datos. Por tanto eliminaremos solo los NAs que representan un porcentaje bajo del total de esa variable (hasta 5%). Los demás NAs los rellenaremos con la moda de esa columna.

In [15]:
#rellenamos los NAs con la media de su respectiva columna:
columnas_a_cambiar = ["P_SAFE", "C_RCFG", "V_YEAR", "C_CONF", "C_RALN", "P_AGE", "P_ISEV", "C_TRAF"]
for i in columnas_a_cambiar:
 c_sev_df[i] = c_sev_df[i].fillna( int(c_sev_df[i].mode() ))


In [16]:
identifica_NA(c_sev_df).head(7)

,Number of NA,Percent NA
V_TYPE,283111,4.83
P_SEX,249740,4.26
C_RSUR,248668,4.24
P_USER,175586,3.00
C_WTHR,102988,1.76
P_PSN,97677,1.67
C_HOUR,59409,1.01


Eliminamos las demás filas con NAs:

In [17]:
c_sev_df =  c_sev_df.dropna()

In [18]:
#codificamos la variable 'P_SEX'
c_sev_df["P_SEX"] = c_sev_df["P_SEX"].replace({"F":"1", "M":"2"})

In [19]:
print("Nuestro dataframe para el Target Fallecimiento tiene la forma:", c_sev_df.shape)
c_sev_df.head(3)

Nuestro dataframe para el Target Fallecimiento tiene la forma: (4996219, 20)


,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,V_YEAR,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,target_c_sev
0,1999,1.0,1.0,20.0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,6.0,1990.0,2,41.0,11.0,1.0,2.0,1.0,0
1,1999,1.0,1.0,20.0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,1.0,1987.0,2,19.0,11.0,1.0,2.0,1.0,0
2,1999,1.0,1.0,20.0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,1.0,1987.0,1,20.0,13.0,2.0,2.0,2.0,0


In [20]:
c_sev_df.dtypes

C_YEAR            int64
C_MNTH          float64
C_WDAY          float64
C_HOUR          float64
C_VEHS          float64
C_CONF          float64
C_RCFG          float64
C_WTHR          float64
C_RSUR          float64
C_RALN          float64
C_TRAF          float64
V_TYPE          float64
V_YEAR          float64
P_SEX            object
P_AGE           float64
P_PSN           float64
P_ISEV          float64
P_SAFE          float64
P_USER          float64
target_c_sev      int64
dtype: object

In [21]:
#Nos aseguramos de que no hay mas valores NA:
identifica_NA(c_sev_df).head()

,Number of NA,Percent NA
C_YEAR,0,0.0
C_MNTH,0,0.0
P_USER,0,0.0
P_SAFE,0,0.0
P_ISEV,0,0.0


<br>

### Tratamiento de variables categoricas

In [24]:
#convertimos a strings variables categoricas codificadas como float. esto nos permitirá hacer el encoding. 
cols = ['C_VEHS','C_CONF','C_RCFG','C_WTHR','C_RSUR','C_RALN','C_TRAF','V_TYPE','P_PSN','P_ISEV','P_SAFE','P_USER']
for i in cols:
 c_sev_df[i] = c_sev_df[i].apply(str)

In [25]:
c_sev_df.dtypes

C_YEAR            int64
C_MNTH          float64
C_WDAY          float64
C_HOUR          float64
C_VEHS           object
C_CONF           object
C_RCFG           object
C_WTHR           object
C_RSUR           object
C_RALN           object
C_TRAF           object
V_TYPE           object
V_YEAR          float64
P_SEX            object
P_AGE           float64
P_PSN            object
P_ISEV           object
P_SAFE           object
P_USER           object
target_c_sev      int64
dtype: object

In [27]:
c_sev_df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,V_YEAR,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,target_c_sev
0,1999,1.0,1.0,20.0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,6.0,1990.0,2,41.0,11.0,1.0,2.0,1.0,0
1,1999,1.0,1.0,20.0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,1.0,1987.0,2,19.0,11.0,1.0,2.0,1.0,0
2,1999,1.0,1.0,20.0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,1.0,1987.0,1,20.0,13.0,2.0,2.0,2.0,0
3,1999,1.0,1.0,8.0,1.0,1.0,2.0,5.0,3.0,6.0,18.0,1.0,1986.0,2,46.0,11.0,1.0,2.0,1.0,0
5,1999,1.0,1.0,17.0,3.0,21.0,2.0,1.0,2.0,1.0,1.0,1.0,1984.0,2,28.0,11.0,1.0,2.0,1.0,0


In [28]:
#dividimos el dataframe en variables categoricas y numericas
df_object = c_sev_df.select_dtypes(include='object')
df_numeric = c_sev_df.select_dtypes(exclude='object')

In [29]:
df_object.head()

,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,P_SEX,P_PSN,P_ISEV,P_SAFE,P_USER
0,2.0,34.0,2.0,1.0,5.0,3.0,3.0,6.0,2,11.0,1.0,2.0,1.0
1,2.0,34.0,2.0,1.0,5.0,3.0,3.0,1.0,2,11.0,1.0,2.0,1.0
2,2.0,34.0,2.0,1.0,5.0,3.0,3.0,1.0,1,13.0,2.0,2.0,2.0
3,1.0,1.0,2.0,5.0,3.0,6.0,18.0,1.0,2,11.0,1.0,2.0,1.0
5,3.0,21.0,2.0,1.0,2.0,1.0,1.0,1.0,2,11.0,1.0,2.0,1.0


In [30]:
df_numeric.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,V_YEAR,P_AGE,target_c_sev
0,1999,1.0,1.0,20.0,1990.0,41.0,0
1,1999,1.0,1.0,20.0,1987.0,19.0,0
2,1999,1.0,1.0,20.0,1987.0,20.0,0
3,1999,1.0,1.0,8.0,1986.0,46.0,0
5,1999,1.0,1.0,17.0,1984.0,28.0,0


In [31]:
#hacemos el encoding. vamos a crear una nueva columna para cada posible valor en cada variable categórica
df_object_dummies = pd.get_dummies(df_object, drop_first=True)

In [32]:
df_object_dummies.shape

(4996219, 144)

In [33]:
df_object_dummies.head(3)

,C_VEHS_10.0,C_VEHS_11.0,C_VEHS_12.0,C_VEHS_13.0,C_VEHS_14.0,C_VEHS_15.0,C_VEHS_16.0,C_VEHS_17.0,C_VEHS_18.0,C_VEHS_19.0,...,P_ISEV_2.0,P_ISEV_3.0,P_SAFE_10.0,P_SAFE_12.0,P_SAFE_13.0,P_SAFE_2.0,P_SAFE_9.0,P_USER_2.0,P_USER_4.0,P_USER_5.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


Hay muchas columnas pero mas adelante haremos seleccion de variables para 'coger' la información importante

A continuación unimos los dataframes

In [34]:
final_df = pd.concat([df_object_dummies,df_numeric],axis=1)

In [35]:
final_df.head(3)

,C_VEHS_10.0,C_VEHS_11.0,C_VEHS_12.0,C_VEHS_13.0,C_VEHS_14.0,C_VEHS_15.0,C_VEHS_16.0,C_VEHS_17.0,C_VEHS_18.0,C_VEHS_19.0,...,P_USER_2.0,P_USER_4.0,P_USER_5.0,C_YEAR,C_MNTH,C_WDAY,C_HOUR,V_YEAR,P_AGE,target_c_sev
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1999,1.0,1.0,20.0,1990.0,41.0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1999,1.0,1.0,20.0,1987.0,19.0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1999,1.0,1.0,20.0,1987.0,20.0,0


In [36]:
final_df.shape

(4996219, 151)

In [37]:
final_df.dtypes.value_counts()

uint8      144
float64      5
int64        2
dtype: int64

In [38]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996219 entries, 0 to 5860380
Columns: 151 entries, C_VEHS_10.0 to target_c_sev
dtypes: float64(5), int64(2), uint8(144)
memory usage: 991.1 MB


In [39]:
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996219 entries, 0 to 5860380
Data columns (total 151 columns):
 #    Column        Dtype  
---   ------        -----  
 0    C_VEHS_10.0   uint8  
 1    C_VEHS_11.0   uint8  
 2    C_VEHS_12.0   uint8  
 3    C_VEHS_13.0   uint8  
 4    C_VEHS_14.0   uint8  
 5    C_VEHS_15.0   uint8  
 6    C_VEHS_16.0   uint8  
 7    C_VEHS_17.0   uint8  
 8    C_VEHS_18.0   uint8  
 9    C_VEHS_19.0   uint8  
 10   C_VEHS_2.0    uint8  
 11   C_VEHS_20.0   uint8  
 12   C_VEHS_21.0   uint8  
 13   C_VEHS_22.0   uint8  
 14   C_VEHS_23.0   uint8  
 15   C_VEHS_24.0   uint8  
 16   C_VEHS_25.0   uint8  
 17   C_VEHS_26.0   uint8  
 18   C_VEHS_27.0   uint8  
 19   C_VEHS_28.0   uint8  
 20   C_VEHS_29.0   uint8  
 21   C_VEHS_3.0    uint8  
 22   C_VEHS_30.0   uint8  
 23   C_VEHS_31.0   uint8  
 24   C_VEHS_32.0   uint8  
 25   C_VEHS_33.0   uint8  
 26   C_VEHS_34.0   uint8  
 27   C_VEHS_35.0   uint8  
 28   C_VEHS_36.0   uint8  
 29   C_VEHS_37.0 

<br>

### Detección de Outliers

Se considera que no hay outliers para este dataset. Los outliers amenudo representan errores en el proceso de recoleccion de los datos. Sin embargo los datos de nuestro dataset representan variables categóricas codificadas y por tanto los valores que pueden tomar estas variables estan limitadas a un rango establecido. En conclusión, es muy poco probable que los valores de nuestras variables representen valores atípicos.

<br>

#### Una vez ya tenemos los dataframes preparados procedemos a cargarlos a un csv para usarlos mas adelante:

In [40]:
#guardamos el final_df como parquet para facilitar el procesamiento
c_sev_df.to_csv('data/c_sev_df.csv')
final_df.to_parquet('data/c_sev_df_encoded.parquet')